# 경찰청 주소를 지도에 나타내기

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import folium
from urllib.parse import quote
import json
import requests

In [2]:
# kakao key 가져오기
with open(r'C:\workspace\api_key\kakao_key.txt') as file:
    kakao_key = file.read()

In [3]:
url = 'https://www.police.go.kr/user/bbs/BD_selectBbsList.do?q_bbsCode=1038&q_tab=1'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

## 1. 서울지역 경찰서로 정보확인

In [4]:
# 전국경찰관서 중 서울 클릭
driver.find_element(By.ID, 'tab_1').click()
time.sleep(3)

In [5]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [6]:
lis = soup.select('tbody > tr')

In [7]:
# 서울 지역의 관서 개수 확인
len(lis)

32

In [8]:
# 가장 마지막에 위치한 관서로 정보 확인 - 서울혜화경찰서
li = lis[-1]

In [9]:
li

<tr>
<td class="show-col">서울혜화경찰서</td>
<td class="show-col">서울시 종로구 창경궁로 112-16 </td>
<td class="show-col">
<a href="https://www.smpa.go.kr/hh/" target="_blank" title="새창열림">https://www.smpa.go.kr/hh/</a>
</td>
</tr>

In [10]:
# 경찰서 이름
title = li.select('.show-col')[0].text
title

'서울혜화경찰서'

In [11]:
# 경찰서 주소
addr = li.select('.show-col')[1].text
addr

'서울시 종로구 창경궁로 112-16 '

In [12]:
# 경찰서 홈페이지 주소
href = li.select('.show-col')[2].text.replace('\n','')
href

'https://www.smpa.go.kr/hh/'

In [13]:
# kakao key를 이용해서 주소로 위도, 경도 구하기
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
headers = {"Authorization": "KakaoAK " + kakao_key}
url = f'{local_url}?query={quote(addr)}'
result = requests.get(url, headers = headers).json()['documents']

float(result[0]['y']), float(result[0]['x'])

(37.5717676404547, 126.999273268519)

In [14]:
# 서울지역 관서들 가져오기
lines = []

for li in lis:
    title = li.select('.show-col')[0].text
    addr = li.select('.show-col')[1].text
    href = li.select('.show-col')[2].text

    lines.append([title, addr, href])

In [15]:
df = pd.DataFrame(lines, columns=['관서명', '주소', '홈페이지'])
df

,관서명,주소,홈페이지
0,서울특별시경찰청,서울시 종로구 사직로8길 31,\nhttps://www.smpa.go.kr/\n
1,서울강남경찰서,서울시 강남구 테헤란로 114길 11,\nhttps://www.smpa.go.kr/gn/\n
2,서울강동경찰서,서울시 강동구 성내로 57,\nhttps://www.smpa.go.kr/gd/\n
3,서울강북경찰서,서울시 강북구 오패산로 406,\nhttps://www.smpa.go.kr/gb/\n
4,서울강서경찰서,서울시 강서구 화곡로 308,\nhttps://www.smpa.go.kr/gs/\n
5,서울관악경찰서,서울시 관악구 관악로5길 33,\nhttps://www.smpa.go.kr/ga/\n
6,서울광진경찰서,서울시 광진구 자양로 167,\nhttps://www.smpa.go.kr/gj/\n
7,서울구로경찰서,서울시 구로구 가마산로 235,\nhttps://www.smpa.go.kr/gr/\n
8,서울금천경찰서,서울시 금천구 시흥대로73길 50,\nhttps://www.smpa.go.kr/gc/\n
9,서울남대문경찰서,서울시 중구 한강대로 410,\nhttps://www.smpa.go.kr/ndm/\n


In [16]:
# kakao_key를 통해서 서울지역 관서들의 위도 경도 좌표 구하기
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
headers = {"Authorization": "KakaoAK " + kakao_key}

lngs, lats = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers = headers).json()['documents']
    try:
        lngs.append(float(result[0]['x']))
        lats.append(float(result[0]['y']))
    except:
        print(df.관서명[i]) # 만약 일치하는 주소가 없으면 관서명 프린트

서울종암경찰서


In [17]:
# '서울종암경찰서' 주소 확인
df[df.관서명.isin(['서울종암경찰서'])].주소

28    서울시 성북구 종암로 135
Name: 주소, dtype: object

In [18]:
# kakao에 주소가 있는지 확인
url = f"{local_url}?query={quote('서울시 성북구 종암로 135')}"
result = requests.get(url, headers=headers).json()
result

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [19]:
# 관서명으로 검색
local_url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
headers = {"Authorization": "KakaoAK " + kakao_key}
url = f'{local_url}?query={quote("서울종암경찰서")}'
result = requests.get(url, headers = headers).json()['documents']

result

[{'address_name': '서울 성북구 하월곡동 27-5',
  'category_group_code': 'PO3',
  'category_group_name': '공공기관',
  'category_name': '사회,공공기관 > 행정기관 > 경찰서',
  'distance': '',
  'id': '12803852',
  'phone': '182',
  'place_name': '서울종암경찰서',
  'place_url': 'http://place.map.kakao.com/12803852',
  'road_address_name': '서울 성북구 화랑로7길 32',
  'x': '127.04013000033655',
  'y': '37.603700560962736'},
 {'address_name': '서울 성북구 종암동 3-1260',
  'category_group_code': 'PO3',
  'category_group_name': '공공기관',
  'category_name': '사회,공공기관 > 행정기관 > 경찰서',
  'distance': '',
  'id': '572822781',
  'phone': '182',
  'place_name': '서울종암경찰서 (2024년 신축예정)',
  'place_url': 'http://place.map.kakao.com/572822781',
  'road_address_name': '',
  'x': '127.032265443516',
  'y': '37.6020199230987'},
 {'address_name': '서울 성북구 종암동 25-47',
  'category_group_code': '',
  'category_group_name': '',
  'category_name': '사회,공공기관 > 행정기관부속시설',
  'distance': '',
  'id': '27540536',
  'phone': '02-3396-7633',
  'place_name': '서울종암경찰서 종암교통센터',

In [20]:
# 주소가 없는경우 관서명으로 검색 후 위도, 경도 구하기
headers = {"Authorization": "KakaoAK " + kakao_key}

lngs, lats = [], []
for i in df.index:
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={quote(df.주소[i])}'
    result = requests.get(url, headers = headers).json()['documents']
    try:
        lngs.append(float(result[0]['x']))
        lats.append(float(result[0]['y']))
    except:
        try:
            url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={quote(df.관서명[i])}'
            result = requests.get(url, headers = headers).json()['documents']
            lngs.append(float(result[0]['x']))
            lats.append(float(result[0]['y']))
        except:
            print(df.관서명[i])

In [21]:
df['위도'] = lats
df['경도'] = lngs
df.head()

,관서명,주소,홈페이지,위도,경도
0,서울특별시경찰청,서울시 종로구 사직로8길 31,\nhttps://www.smpa.go.kr/\n,37.574946,126.971980
1,서울강남경찰서,서울시 강남구 테헤란로 114길 11,\nhttps://www.smpa.go.kr/gn/\n,37.509367,127.067100
2,서울강동경찰서,서울시 강동구 성내로 57,\nhttps://www.smpa.go.kr/gd/\n,37.528678,127.126957
3,서울강북경찰서,서울시 강북구 오패산로 406,\nhttps://www.smpa.go.kr/gb/\n,37.637170,127.027042
4,서울강서경찰서,서울시 강서구 화곡로 308,\nhttps://www.smpa.go.kr/gs/\n,37.551558,126.849656


## 2. 전국의 경찰서 정보 가져오기

In [22]:
# 전국 지역의 수
count= len(soup.select('.nav.nav-tabs.ui-multi-line > li'))
count

18

In [23]:
lines = []
for i in range(1, count + 1):
    # 페이지 이동
    driver.find_element(By.ID, f'tab_{i}').click()
    time.sleep(3)
    
    # 각 페이지의 데이터 추출
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('tbody > tr')
    
    for li in lis:
        title = li.select('.show-col')[0].text
        addr = li.select('.show-col')[1].text
        href = li.select('.show-col')[2].text

        lines.append([title, addr, href])

In [24]:
df = pd.DataFrame(lines, columns=['관서명', '주소', '홈페이지'])
df

,관서명,주소,홈페이지
0,서울특별시경찰청,서울시 종로구 사직로8길 31,\nhttps://www.smpa.go.kr/\n
1,서울강남경찰서,서울시 강남구 테헤란로 114길 11,\nhttps://www.smpa.go.kr/gn/\n
2,서울강동경찰서,서울시 강동구 성내로 57,\nhttps://www.smpa.go.kr/gd/\n
3,서울강북경찰서,서울시 강북구 오패산로 406,\nhttps://www.smpa.go.kr/gb/\n
4,서울강서경찰서,서울시 강서구 화곡로 308,\nhttps://www.smpa.go.kr/gs/\n
...,...,...,...
272,합천경찰서,경남 합천군 합천읍 황강체육공원로 67,\nhttps://www.gnpolice.go.kr/hc\n
273,제주특별자치도경찰청,제주시 수목원 서길 37,\nhttps://www.jjpolice.go.kr\n
274,서귀포경찰서,서귀포시 신중로 27,\nhttps://www.jjpolice.go.kr/seogwipo\n
275,제주동부경찰서,제주시 동광로 66,\nhttps://www.jjpolice.go.kr/dongbu\n


In [25]:
driver.close()

In [26]:
# 주소가 없는경우 관서명으로 검색 후 위도, 경도 구하기
headers = {"Authorization": "KakaoAK " + kakao_key}

lngs, lats = [], []
for i in df.index:
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={quote(df.주소[i])}'
    result = requests.get(url, headers = headers).json()['documents']
    try:
        lngs.append(float(result[0]['x']))
        lats.append(float(result[0]['y']))
    except:
        try:
            url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={quote(df.관서명[i])}'
            result = requests.get(url, headers = headers).json()['documents']
            lngs.append(float(result[0]['x']))
            lats.append(float(result[0]['y']))
        except:
            print(df.관서명[i])

In [27]:
# df에 위도, 경도를 추가
df['위도'] = lats
df['경도'] = lngs
df.head()

,관서명,주소,홈페이지,위도,경도
0,서울특별시경찰청,서울시 종로구 사직로8길 31,\nhttps://www.smpa.go.kr/\n,37.574946,126.971980
1,서울강남경찰서,서울시 강남구 테헤란로 114길 11,\nhttps://www.smpa.go.kr/gn/\n,37.509367,127.067100
2,서울강동경찰서,서울시 강동구 성내로 57,\nhttps://www.smpa.go.kr/gd/\n,37.528678,127.126957
3,서울강북경찰서,서울시 강북구 오패산로 406,\nhttps://www.smpa.go.kr/gb/\n,37.637170,127.027042
4,서울강서경찰서,서울시 강서구 화곡로 308,\nhttps://www.smpa.go.kr/gs/\n,37.551558,126.849656


In [28]:
df.to_csv('전국경찰관서.csv', index=False)

## 3. 지도에 표시하기

In [29]:
map = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=8)
for i in df.index[:]:
    folium.Marker(
        location=[df.위도[i],df.경도[i]], 
        popup=folium.Popup(df.주소[i], max_width=200), 
        tooltip=df.관서명[i],
        icon = folium.Icon(icon = 'glyphicon glyphicon-star')
    ).add_to(map)
title = '<h3 align="center" style="font-size:20px">전국 경찰관서 위치</h3>'
map.get_root().html.add_child(folium.Element(title))
map